## Data Cleaning  ##

Going to reduce the number of files being used and will clean and merge them w/in this document.  Goal is to have data organized so that we can work with the following:
    - genre (TMDB)
    - studio (gross profits)
    - cost (TN, not sure what that stands for)
    - profit (BOM)
    

In [1]:
!ls zippedData

InitialDataExploration.ipynb
Untitled.ipynb
bom.movie_gross.csv.gz
imdb.name.basics.csv.gz
imdb.title.akas.csv.gz
imdb.title.basics.csv.gz
imdb.title.crew.csv.gz
imdb.title.principals.csv.gz
imdb.title.ratings.csv.gz
rt.movie_info.tsv.gz
rt.reviews.tsv.gz
tmdb.movies.csv.gz
tn.movie_budgets.csv.gz


In [14]:
import pandas as pd

movie_gross_df = pd.read_csv('zippedData/bom.movie_gross.csv.gz')
# imdb_name_basics_df = pd.read_csv('zippedData/imdb.name.basics.csv.gz')
# imdb_title_akas_df = pd.read_csv('zippedData/imdb.title.akas.csv.gz')
imdb_title_basics_df = pd.read_csv('zippedData/imdb.title.basics.csv.gz')
# imdb_title_crew_df = pd.read_csv('zippedData/imdb.title.crew.csv.gz')
# imdb_title_principals_df = pd.read_csv('zippedData/imdb.title.principals.csv.gz')
imdb_title_ratings_df = pd.read_csv('zippedData/imdb.title.ratings.csv.gz')
# rt_movie_info_df = pd.read_csv('zippedData/rt.movie_info.tsv.gz', delimiter='\t')
# rt_reviews_df = pd.read_csv('zippedData/rt.reviews.tsv.gz', delimiter = '\t', encoding='latin1')
tmdb_movies_df = pd.read_csv('zippedData/tmdb.movies.csv.gz')
tn_movie_budgets_df = pd.read_csv('zippedData/tn.movie_budgets.csv.gz')

## Supress scientific notation ##

In [19]:
##will display floats to the second decimal place
## this code resets this change: pd.reset_option('^display.', silent=True)

pd.options.display.float_format = '{:.2f}'.format

In [3]:
movie_gross_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3387 entries, 0 to 3386
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   title           3387 non-null   object 
 1   studio          3382 non-null   object 
 2   domestic_gross  3359 non-null   float64
 3   foreign_gross   2037 non-null   object 
 4   year            3387 non-null   int64  
dtypes: float64(1), int64(1), object(3)
memory usage: 132.4+ KB


### Cleaning To Dos for Movie Gross ###
- Get rid of rows with null values in the studio field and in the domestic gross field since there aren't many of them 
- ignore the foreign gross nulls, info may be listed on other sheets iirc. Alternately possibly could be calculated w/ info from other sheets
- do a unique value check on year, make sure nothing weird
- do a unique value check on studio, can we replace w/proper names?
- sort by domestic gross and check the tail
- clean out "(YEAR)" from movie titles (via regular expression?)

In [4]:
movie_gross_df.dropna(subset=['studio','domestic_gross'], inplace = True)

In [5]:
movie_gross_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3356 entries, 0 to 3386
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   title           3356 non-null   object 
 1   studio          3356 non-null   object 
 2   domestic_gross  3356 non-null   float64
 3   foreign_gross   2007 non-null   object 
 4   year            3356 non-null   int64  
dtypes: float64(1), int64(1), object(3)
memory usage: 157.3+ KB


In [6]:
movie_gross_df['year'].unique()

array([2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018], dtype=int64)

**NOTE THAT THIS ONLY RUNS FROM 2010-2018**

In [7]:
movie_gross_df['studio'].unique()

array(['BV', 'WB', 'P/DW', 'Sum.', 'Par.', 'Uni.', 'Fox', 'Wein.', 'Sony',
       'FoxS', 'SGem', 'WB (NL)', 'LGF', 'MBox', 'CL', 'W/Dim.', 'CBS',
       'Focus', 'MGM', 'Over.', 'Mira.', 'IFC', 'CJ', 'NM', 'SPC', 'ParV',
       'Gold.', 'JS', 'RAtt.', 'Magn.', 'Free', '3D', 'UTV', 'Rela.',
       'Zeit.', 'Anch.', 'PDA', 'Lorb.', 'App.', 'Drft.', 'Osci.', 'IW',
       'Rog.', 'Eros', 'Relbig.', 'Viv.', 'Hann.', 'Strand', 'NGE',
       'Scre.', 'Kino', 'Abr.', 'CZ', 'ATO', 'First', 'GK', 'FInd.',
       'NFC', 'TFC', 'Pala.', 'Imag.', 'NAV', 'Arth.', 'CLS', 'Mont.',
       'Olive', 'CGld', 'FOAK', 'IVP', 'Yash', 'ICir', 'FM', 'Vita.',
       'WOW', 'Truly', 'Indic.', 'FD', 'Vari.', 'TriS', 'ORF', 'IM',
       'Elev.', 'Cohen', 'NeoC', 'Jan.', 'MNE', 'Trib.', 'Rocket',
       'OMNI/FSR', 'KKM', 'Argo.', 'SMod', 'Libre', 'FRun', 'WHE', 'P4',
       'KC', 'SD', 'AM', 'MPFT', 'Icar.', 'AGF', 'A23', 'Da.', 'NYer',
       'Rialto', 'DF', 'KL', 'ALP', 'LG/S', 'WGUSA', 'MPI', 'RTWC', 'FIP',
  

We are not going to try to replace these with proper names, too many - what may end up happening is that we look at the top few studios and just correct those...

In [8]:
v_count = movie_gross_df['studio'].value_counts()

In [9]:
v_count[:11]

IFC      166
Uni.     147
WB       140
Fox      136
Magn.    136
SPC      123
Sony     109
BV       106
LGF      102
Par.     101
Eros      89
Name: studio, dtype: int64

In [10]:
movie_gross_df.loc[movie_gross_df['studio'] == "LGF"] #plug in different studio values to see movie titles 

,title,studio,domestic_gross,foreign_gross,year
26,The Expendables,LGF,103100000.0,171400000,2010
51,Saw 3D,LGF,45700000.0,90400000,2010
64,Killers,LGF,47100000.0,51100000,2010
65,Kick-Ass,LGF,48100000.0,48100000,2010
87,The Last Exorcism,LGF,41000000.0,26700000,2010
...,...,...,...,...,...
3207,Hell Fest,LGF,11100000.0,7100000,2018
3229,Kin,LGF,5700000.0,4300000,2018
3231,Traffik,LGF,9200000.0,336000,2018
3235,Condorito: La Pelicula,LGF,448000.0,8000000,2018


In [11]:
movie_gross_df.sort_values('domestic_gross').head(10)

,title,studio,domestic_gross,foreign_gross,year
1476,Storage 24,Magn.,100.0,NaN,2013
2321,The Chambermaid,FM,300.0,NaN,2015
2757,Satanic,Magn.,300.0,NaN,2016
2756,News From Planet Mars,KL,300.0,NaN,2016
1018,Apartment 143,Magn.,400.0,426000,2012
3078,2:22,Magn.,400.0,NaN,2017
3077,Max & Leon,Distrib.,500.0,NaN,2017
1126,Death of a Superhero,Trib.,600.0,NaN,2012
2920,Amityville: The Awakening,W/Dim.,700.0,7700000,2017
1475,Into the White,Magn.,700.0,NaN,2013


looks like domestic gross makes sense, both highs and lows 

In [12]:
#importing regex so I can more easily find the films with the year added to their titles

import re

In [26]:
reg_expression = '\([0-9]{4}\)' #looks for 4-digit numeric string between '(' and ')'

# found the below format online, returns all movies that meet the reg expression

titles_need_formatting = movie_gross_df[movie_gross_df['title'].str.count(reg_expression)>0]
titles_need_formatting

,title,studio,domestic_gross,foreign_gross,year
1,Alice in Wonderland (2010),BV,334200000.00,691300000,2010
10,Clash of the Titans (2010),WB,163200000.00,330000000,2010
55,A Nightmare on Elm Street (2010),WB (NL),63100000.00,52600000,2010
85,Legion (2010),SGem,40200000.00,27800000,2010
106,Death at a Funeral (2010),SGem,42700000.00,6300000,2010
...,...,...,...,...,...
3326,The Little Mermaid (2018),Conglomerate,147000.00,NaN,2018
3340,Revenge (2018),Neon,102000.00,NaN,2018
3341,Unstoppable (2018),WGUSA,101000.00,NaN,2018
3365,The Apparition (2018),MBox,28300.00,NaN,2018


In [27]:
## export the full data frame to a csv file so I could quickly visually confirm that 
## the results all had the addn'l year info at the end of the title string 
## (they did)

titles_need_formatting.to_csv("titles.csv")



In [38]:
# removes the last 7 chars in a title string if it meets the reg expression
# new_string = re.sub(r"xxx|yyy", "abc", a_strin

movie_gross_df['title'] = movie_gross_df['title'].apply(lambda x: re.sub(reg_expression,"",x)).str.rstrip()

# movie_gross_df[movie_gross_df['title'].apply(lambda x: re.sub(reg_expression,"",x))]

In [40]:
#this cell confirms that all the TITLE (YEAR) values in the title column have 
#had the year info removed. 

movie_gross_df[movie_gross_df['title'].str.count(reg_expression)>0]

,title,studio,domestic_gross,foreign_gross,year


In [41]:
# finds all the re-releases

substring = 're-release'

movie_gross_df[movie_gross_df['title'].str.find(substring)>0]

,title,studio,domestic_gross,foreign_gross,year
1823,The Conformist (2014 re-release),KL,58700.00,NaN,2014
1833,Alphaville (2013 re-release),Rialto,47700.00,NaN,2014
2139,The Third Man (2015 re-release),Rialto,449000.00,NaN,2015
2604,Only Yesterday (2016 re-release),GK,453000.00,NaN,2016
3264,2001: A Space Odyssey (2018 re-release),WB,3200000.00,NaN,2018
3289,Schindler's List (2018 re-release),Uni.,833000.00,NaN,2018
3296,The Sound of Music (2018 re-release),Fathom,616000.00,NaN,2018
3383,Edward II (2018 re-release),FM,4800.00,NaN,2018


In [54]:
#drop the re-releases from the data set as they were not made in the listed year

movie_gross_df.drop(movie_gross_df[movie_gross_df['title'].str.find(substring)>0].index, inplace=True)

In [55]:
#check to make sure re-releases are gone from df

movie_gross_df[movie_gross_df['title'].str.find(substring)>0]

,title,studio,domestic_gross,foreign_gross,year


In [62]:
#Clean up column titles 


movie_gross_df.rename(columns = {'title':'Title', 'studio':'Studio', 'domestic_gross':'Domestic Gross',
          'foreign_gross':'Foreign Gross', 'year':'Year'}, inplace = True)
movie_gross_df

,Title,Studio,Domestic Gross,Foreign Gross,Year
0,Toy Story 3,BV,415000000.00,652000000,2010
1,Alice in Wonderland,BV,334200000.00,691300000,2010
2,Harry Potter and the Deathly Hallows Part 1,WB,296000000.00,664300000,2010
3,Inception,WB,292600000.00,535700000,2010
4,Shrek Forever After,P/DW,238700000.00,513900000,2010
...,...,...,...,...,...
3381,Beauty and the Dogs,Osci.,8900.00,NaN,2018
3382,The Quake,Magn.,6200.00,NaN,2018
3384,El Pacto,Sony,2500.00,NaN,2018
3385,The Swan,Synergetic,2400.00,NaN,2018


## Cleaning To Dos for tmdb_movies_df ##
- add genre columns
- check min/maxs
- note no nulls
- clean out rows w/empty lists in genre category (like 10% of the data set, 2.6k, more then I like but average isn't going to cut it and there's no way to manually enter that many)

In [ ]:
tmdb_movies_df.tail()

In [ ]:
tmdb_movies_df.info()

### The below gets rid of the empty lists in genre ###

In [ ]:
tmdb_movies_df[tmdb_movies_df['genre_ids'] == '[]']

In [ ]:

tmdb_movies_df.drop(tmdb_movies_df[tmdb_movies_df['genre_ids'] == '[]'].index, inplace = True)

In [ ]:
tmdb_movies_df.info()

In [ ]:
tmdb_movies_df[tmdb_movies_df['genre_ids'] == '[]']

In [ ]:
tmdb_movies_df.describe()

In [ ]:
## going to try and create a column for each genre based on the list and then populate it with boolean for the particular film
## start going to make a dictionary of the codes, then going to loop through, each loop creates a column and populates it

genre_dict = {28:'Action', 12: 'Adventure', 16: 'Animation', 35: 'Comedy', 80:'Crime', 99:'Documentary',18:'Drama', 10751:'Family',
              14:'Fantasy', 36: 'History', 27:'Horror', 10402:'Music', 9648:'Mystery', 10749:'Romance', 878:'Science Fiction',
              53:'Thriller', 10752:'War',37:'Western'}
#didn't include TV Movie category since we don't care about those (may want to filter them out...)

In [ ]:
tmdb_movies_df[tmdb_movies_df['genre_ids'].str.contains('10770')]

## Clears out TV Movies from df, creates a new column for each genre fills it with true/false based on genre id column ##

In [ ]:
# clears out the tv movies 
tmdb_movies_df.drop(tmdb_movies_df.loc[tmdb_movies_df['genre_ids'].str.contains('10770')].index, inplace = True)

In [ ]:
tmdb_movies_df['genre_ids'].str.contains('28')

In [ ]:
for key, value in genre_dict.items():
    tmdb_movies_df[value] = tmdb_movies_df['genre_ids'].str.contains(str(key))

In [ ]:
tmdb_movies_df.head()

### Cleaning Up imdb_title_ratings_df ###

In [15]:
imdb_title_ratings_df.head()

,tconst,averagerating,numvotes
0,tt10356526,8.3,31
1,tt10384606,8.9,559
2,tt1042974,6.4,20
3,tt1043726,4.2,50352
4,tt1060240,6.5,21


In [20]:
imdb_title_ratings_df.describe()

,averagerating,numvotes
count,73856.00,73856.00
mean,6.33,3523.66
std,1.47,30294.02
min,1.00,5.00
25%,5.50,14.00
50%,6.50,49.00
75%,7.40,282.00
max,10.00,1841066.00


These numbers seem okay - nothing improbable

In [21]:
imdb_title_ratings_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 73856 entries, 0 to 73855
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   tconst         73856 non-null  object 
 1   averagerating  73856 non-null  float64
 2   numvotes       73856 non-null  int64  
dtypes: float64(1), int64(1), object(1)
memory usage: 1.7+ MB
